In [1]:
import pandas as pd
import torch
import numpy as np
import random
from random import shuffle
np.random.seed(17)
random.seed(17)
torch.manual_seed(17)
import wget
import os
import aiohttp
import asyncio
from fpl import FPL
from torch.utils.data import TensorDataset, DataLoader
from player import Player
from team import Team
from data_processor import get_fpl, get_players, get_teams, get_training_datasets, get_all_player_features
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
fpl = await get_fpl()
team_feature_names = ["npxGA"]
teams = get_teams(team_feature_names=team_feature_names, visualize=False)
player_feature_names = ["total_points"]
players = await get_players(player_feature_names, team_feature_names, visualize=False, num_players=590)

In [9]:
def get_timeseries_dataset(players, context_window=10):
    '''
        Args
            players - List of players 
        Returns
            Time series dataset containing player points

            context_window = feature_window + 1 + choice    
            X - shape (N, context_window)
    '''
    timeseries_dataset = []
    for player in players:
        player_points = player.player_features[0,:]
        for i in range(len(player_points) - context_window):
            timeseries_dataset.append(player_points[i:i+context_window])
    random.shuffle(timeseries_dataset)
    return timeseries_dataset


timeseries_dataset = get_timeseries_dataset(players[0:5])
timeseries_dataset

[array([0, 0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=object),
 array([0, 0, 0, 2, 0, 0, 0, 0, 0, 2], dtype=object),
 array([0, 0, 2, 0, 0, 0, 0, 0, 2, 2], dtype=object),
 array([0, 2, 0, 0, 0, 0, 0, 2, 2, 2], dtype=object),
 array([2, 0, 0, 0, 0, 0, 2, 2, 2, 6], dtype=object),
 array([0, 0, 0, 0, 0, 2, 2, 2, 6, 5], dtype=object),
 array([0, 0, 0, 0, 2, 2, 2, 6, 5, 2], dtype=object),
 array([0, 0, 0, 2, 2, 2, 6, 5, 2, 1], dtype=object),
 array([0, 0, 2, 2, 2, 6, 5, 2, 1, 0], dtype=object),
 array([0, 2, 2, 2, 6, 5, 2, 1, 0, 2], dtype=object),
 array([2, 2, 2, 6, 5, 2, 1, 0, 2, 3], dtype=object),
 array([2, 2, 6, 5, 2, 1, 0, 2, 3, 3], dtype=object),
 array([2, 6, 5, 2, 1, 0, 2, 3, 3, 2], dtype=object),
 array([6, 5, 2, 1, 0, 2, 3, 3, 2, 2], dtype=object),
 array([5, 2, 1, 0, 2, 3, 3, 2, 2, 1], dtype=object),
 array([2, 1, 0, 2, 3, 3, 2, 2, 1, 2], dtype=object),
 array([1, 0, 2, 3, 3, 2, 2, 1, 2, 8], dtype=object),
 array([0, 2, 3, 3, 2, 2, 1, 2, 8, 2], dtype=object),
 array([2, 3, 3, 2, 2, 1, 2,